In [ ]:
f = open("hosts.txt")

In [ ]:
nodes = readlines(f)

In [ ]:
using Distributed, LinearAlgebra, Plots, Printf, ApproxFun, PeriodicKdV

In [ ]:
num_procs = 40
addprocs([nodes[2] for j in 1:num_procs],tunnel=true)
addprocs([nodes[3] for j in 1:num_procs],tunnel=true)
addprocs([nodes[4] for j in 1:num_procs],tunnel=true);

In [ ]:
addprocs(num_procs-1);

In [ ]:
prec = 500
sp(x) = BigFloat(x,precision = prec)
spc(x) = BigFloat(real(x),precision = prec)+1im*BigFloat(imag(x),precision = prec)

In [ ]:
@everywhere using LinearAlgebra, Printf, ApproxFun, PeriodicKdV

In [ ]:
ω = pi/sqrt(6.0 |> sp)
T = 2.0*pi/sqrt(6 |> sp)
h = .5;

In [ ]:
Δ1 = λ -> 2*cos((T-ω)*sqrt(λ-h))*cos(ω*sqrt(λ))
Δ2 = λ -> -(2*λ-h)/(sqrt(λ)*sqrt(λ-h))*sin(ω*sqrt(λ))*sin((T-ω)*sqrt(λ-h))
Δ = λ ->  Δ1(λ |> complex) + Δ2(λ |> complex) |> real
T22 = λ -> cos(ω*sqrt(λ))*cos((T-ω)*sqrt(λ-h)) - sqrt(λ-h)/sqrt(λ)*sin((T-ω)*sqrt(λ-h))*sin(ω*sqrt(λ))
T11 = λ -> Δ(λ) - T22(λ)
y2sqrt = λ -> 1/sqrt(λ)*cos((T-ω)*sqrt(λ-h))*sin(ω*sqrt(λ)) + 1/sqrt(λ - h)*cos(ω*sqrt(λ))*sin((T-ω)*sqrt(λ-h))

y2 = λ -> y2sqrt(λ^2)
dy2 = λ -> imag(y2(λ + 1im*spc(1e-50))/sp(1e-50))

In [ ]:
function Newton(f,df,x0,eps,nmax)
    x = x0
    for i = 1:nmax
        δ = -f(x)/df(x)
        if abs(δ) < eps
            return x + δ
        else
            x = x + δ
        end
    end
    if abs(f(x)) > 1e-15
        @warn "failed to converge"
    end
    return x
end

function Bisection(f,A,B,nmax)
    a = A; b = B;
    if abs(f(a)) < 1e-50
        return a
    elseif abs(f(b)) < 1e-50
        return b
    end
    c = a;
    for i = 1:nmax
        fa = f(a); 
        fb = f(b);
        c = (a + b)/2;
        fc = f(c)
        #println((fa,fc,fb))
        #println((a,c,b))
        if fa*fc > 0
            a = c
        else
            b = c
        end
    end
    c
end

In [ ]:
tm = λ -> sqrt(Δ(λ)^2 - 4.0 |> complex) - (T11(λ) - T22(λ))
tp = λ -> sqrt(Δ(λ)^2 - 4.0 |> complex) + (T11(λ) - T22(λ))

target_g = 1000
z1 = Newton(y2,dy2,sqrt(1.5 |> sp ),1e-32,100)
zs = [z1];
for i = 1:target_g
    x0 = zs[end] + pi/T
    x0 = Newton(y2,dy2,x0,1e-50,100)
    zs = vcat(zs,[x0])
end
oldzs = copy(zs)
zs = zs.^2

gs = [Bisection(λ -> Δ(λ) - 2,0.00001 |> sp ,zs[1] |> sp ,60) |> Float64, Bisection(λ -> Δ(λ) + 2,0.00001 |> sp,zs[1] |> sp,60) |> Float64] 
for i = 0:target_g-1
    gs = vcat(gs,[Bisection(λ -> Δ(λ) + 2, zs[i+1] |> sp, zs[i+2] |> sp,60) |> Float64, Bisection(λ -> Δ(λ) - 2,zs[i+1] |> sp,zs[i+2] |> sp,80) |> Float64 ])
end
gs = gs |> sort

α1 = gs[1]
zs = map(Float64,hcat(copy(zs) .- α1,sign.(map(tp,zs) - map(tm,zs) |> real)));
gaps = hcat(gs[2:2:end-1],gs[3:2:end]) .- α1;

In [ ]:
gaps[1:15,:]

In [ ]:
for i = 1:target_g
   if gaps[i,1] <= zs[i,1] <= gaps[i,2]
        
   else
       println(i)
       println( min(abs(gaps[i,1]-zs[i,1]),abs(gaps[i,2]-zs[i,1])))
        @warn "not in the gap"
    end
end

# g = 300 #

In [ ]:
g = 300
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100);
BA = BakerAkhiezerFunction(S,200.;tols=[1e-4,false],iter = 20,max_pts = 10, show_flag = true, choose_points = [10,10,10,10,3]);
# BA = BakerAkhiezerFunction(S,200.;tols=[1e-7,false],iter = 20);
@everywhere BA = $BA
u = (x,t) -> -KdV(BA,x/sqrt(6),t*6^(-3/2),1e-4)

In [ ]:
x = 0:.001:2*pi |> Array
t = 0.0
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p1 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.3,1.],lw=3,label = "", framestyle = :box, fill = (-2,:lightblue))


In [ ]:
savefig(p1,"p0_300.pdf")

In [ ]:
x = 0:.001:2*pi |> Array
t = 0.1*pi
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p1 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.3,1.],lw=3,label = "", framestyle = :box, fill = (-2,:lightblue))


In [ ]:
savefig(p1,"p1_300.pdf")

In [ ]:
x = 0:.001:2*pi |> Array
t = 0.1
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p1 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.3,1.],lw=3,label = "", framestyle = :box, fill = (-2,:lightblue))

In [ ]:
savefig(p1,"p2_300.pdf")

In [ ]:
x = 0:.001:2*pi |> Array
t = 0.5*pi
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p1 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.3,1.],lw=3,label = "", framestyle = :box, fill = (-2,:lightblue))


In [ ]:
savefig(p1,"p3_300.pdf")

In [ ]:
x = 0:.001:2*pi |> Array
t = 0.01*pi
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p1 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.3,1.],lw=3,label = "", framestyle = :box, fill = (-2,:lightblue))


In [ ]:
savefig(p1,"p4_300.pdf")

In [ ]:
x = 0:.001:2*pi |> Array
t = 1.03*pi
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p1 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.3,1.],lw=3,label = "", framestyle = :box, fill = (-2,:lightblue))


In [ ]:
savefig(p1,"p5_300.pdf")

In [ ]:
x = 0:.001:2*pi |> Array
t = .2
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p1 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.3,1.],lw=3,label = "", framestyle = :box, fill = (-2,:lightblue))


In [ ]:
savefig(p1,"p6_300.pdf")

## Increasing genus, zoomed in

In [ ]:
range = 100:200:900
data = zeros(length(range))
i = 1
for g = range
    S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100;cycleflag=true);
    println("Surface constructed")
    BA = BakerAkhiezerFunction(S,200.;tols=[1e-4,false],iter = 20,max_pts = 10, show_flag = true, choose_points = [10,10,10,10,2]);
    println("BA constructed")
    @everywhere BA = $BA
    println("Begin computation")
    u = (x,t) -> -KdV(BA,x/sqrt(6),t*6^(-3/2),1e-4)
    x = 1:0.001:2 |> Array
    U = pmap(x -> u(x,0.1*pi) |> real, x);
    str = @sprintf("test_zoomed_%i.pdf",range[i])
    i += 1
    p1 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.11,-0.08],lw=3,label = "", framestyle = :box, fill = (-2,:lightblue))
    savefig(p1,str)
    display(p1)
end

# Reconstruct IC error

In [ ]:
range = 50:50:600
data = zeros(length(range))
datasum = zeros(length(range))
i = 1
for g = range
    S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100,[true,true]);
    BA = BakerAkhiezerFunction(S,200.,[1e-7,false],20,0,false);
    @everywhere BA = $BA
    u = (x,t) -> -KdV(BA,x/sqrt(6),t*6^(-3/2),1e-4)

    x = 0.1:.01:3|> Array
    U = pmap(x -> u(x,0) |> real, x);
    V = map(x -> x > pi ? 1 : 0,x)
    plot(x,U)
    plot!(x,V,) |> display
    data[i] = maximum(abs.(U-V))
    datasum[i] = sum(abs.(U-V))
    println((g,data[i],datasum[i]))
    i += 1
end

In [ ]:
range = 50:50:700
data = zeros(length(range))
i = 1
for g = range
    S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100,[true,true]);
    BA = BakerAkhiezerFunction(S,200.,[1e-20,false],20,0,false);
    @everywhere BA = $BA
    u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2),1e-4) - 0*h/2

    x = 0.1:.1:3|> Array
    U = pmap(x -> u(x,0) |> real, x);
    V = map(x -> x > pi ? 1 : 0,x)
    data[i] = maximum(abs.(U-V))
    println((g,data[i]))
    i += 1
end

## Coefficient plotting

In [ ]:
g = 200
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100;cycleflag=true);
println("Surface constructed")
BA = BakerAkhiezerFunction(S,200.;tols=[1e-4,false],iter = 20,max_pts = 10, show_flag = true, choose_points = [10,10,10,10,3]);
println("BA constructed")
@everywhere BA = $BA
println("Begin computation")
ba = BA(0.0,0.0);

In [ ]:
A,B = BA(0.0,0.0; getmatrices = true);

In [ ]:
l1 = eigvals(B + A)
p = scatter(l1 |> real, l1 |> imag, label = "Eigenvalues before preconditioning", framestyle = :box)
savefig(p,"before_precond.pdf")

In [ ]:
l1 = eigvals(I + B\A)
p = scatter(l1 |> real, l1 |> imag, label = "Eigenvalues after preconditioning", framestyle = :box)
savefig(p,"after_precond.pdf")

In [ ]:
js = vcat(-g:-1,1:g);

In [ ]:
p = plot(js,abs.([ out.cs[1] for out in ba[2] ]), yaxis = ([0,5]), framestyle = :box, legend = false)
savefig(p,"first_coef.pdf")
display(p)

In [ ]:
p = plot(js,abs.([ out.cs[2] for out in ba[2] ]), yaxis = ([1e-16,.1], :log), framestyle = :box, legend = false,yticks = 10.0 .^(-(1:2:16)))
savefig(p,"second_coef.pdf")
display(p)

In [ ]:
p = plot(js,abs.([ out.cs[3] for out in ba[2] ]), yaxis = ([1e-15,.01], :log), framestyle = :box, legend = false, yticks = 10.0 .^(-(2:2:16)))
savefig(p,"third_coef.pdf")
display(p)

## Old code

In [ ]:
g = 3
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100,[true,true]);
BA = BakerAkhiezerFunction(S,200.,[1e-40,false],20,0,true);
@everywhere BA = $BA
u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2),1e-8) - 0*h/2

x = 0.1:.1:3|> Array
U = pmap(x -> u(x,0) |> real, x);
V = map(x -> x > pi ? 1 : 0,x)
plot(x,U)
plot!(x,V)
print(maximum(abs.(U-V)))

In [ ]:
g = 3
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100,[false,false]); # compare with old method
BA = BakerAkhiezerFunction(S,200.,[1e-40,false],20,0,true);
@everywhere BA = $BA
u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2),1e-8) - 0*h/2

x = 0.1:.1:3|> Array
U = pmap(x -> u(x,0) |> real, x);
V = map(x -> x > pi ? 1 : 0,x)
plot(x,U)
plot!(x,V)
print(maximum(abs.(U-V)))

In [ ]:
g = 30
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100,[true,true]);
BA = BakerAkhiezerFunction(S,200.,[1e-40,false],20,0,true);
@everywhere BA = $BA
u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2),1e-8) - 0*h/2

x = 0.1:.1:3|> Array
U = pmap(x -> u(x,0) |> real, x);
V = map(x -> x > pi ? 1 : 0,x)
plot(x,U)
plot!(x,V)
print(maximum(abs.(U-V)))

In [ ]:
g = 60
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100,[true,true]);
BA = BakerAkhiezerFunction(S,200.,[1e-40,false],20,0,true);
@everywhere BA = $BA
u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2),1e-8) - 0*h/2

x = 0.1:.1:3|> Array
U = pmap(x -> u(x,0) |> real, x);
V = map(x -> x > pi ? 1 : 0,x)
plot(x,U)
plot!(x,V)
print(maximum(abs.(U-V)))

In [ ]:
g = 120
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100,[true,true]);
BA = BakerAkhiezerFunction(S,200.,[1e-40,false],20,0,true);
@everywhere BA = $BA
u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2),1e-8) - 0*h/2

x = 0.1:.1:3|> Array
U = pmap(x -> u(x,0) |> real, x);
V = map(x -> x > pi ? 1 : 0,x)
plot(x,U)
plot!(x,V)
print(maximum(abs.(U-V)))

In [ ]:
g = 240
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100,[true,true]);
BA = BakerAkhiezerFunction(S,200.,[1e-40,false],20,0,true);
@everywhere BA = $BA
u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2),1e-8) - 0*h/2

x = 0.1:.1:3|> Array
U = pmap(x -> u(x,0) |> real, x);
V = map(x -> x > pi ? 1 : 0,x)
plot(x,U)
plot!(x,V)
print(maximum(abs.(U-V)))

In [ ]:
g = 480
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100,[true,true]);
BA = BakerAkhiezerFunction(S,200.,[1e-40,false],20,0,true);
@everywhere BA = $BA
u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2),1e-8) - 0*h/2

x = 0.1:.1:3|> Array
U = pmap(x -> u(x,0) |> real, x);
V = map(x -> x > pi ? 1 : 0,x)
plot(x,U)
plot!(x,V)
print(maximum(abs.(U-V)))

In [ ]:
g = 480
S = HyperellipticSurface(gaps[1:g,:],zs[1:g,:],α1,100,[true,true]);
BA = BakerAkhiezerFunction(S,200.,[1e-40,false],20,0,true);
@everywhere BA = $BA
u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2),1e-8) - 0*h/2

x = 0.1:.1:3|> Array
U = pmap(x -> u(x,0) |> real, x);
V = map(x -> x > pi ? 1 : 0,x)
plot(x,U)
plot!(x,V)
print(maximum(abs.(U-V)))

In [ ]:
x = 0.1:.1:3 |> Array
t = 0.0*pi
U = pmap(x -> u(x,0) |> real, x);
V = map(x -> x > pi ? 1 : 0,x)
plot(x,abs.(U-V))

In [ ]:
S.Ωx*sqrt(6)

In [ ]:
data = zeros(9)
i = 1
for g = 100:100:600
    @everywhere g = $g
    some_zs = hcat(zs[1:g] .- α1,sign.(map(tp,zs[1:g]) - map(tm,zs[1:g]) |> real));
    @everywhere some_zs = $some_zs
    @everywhere gaps = $gaps
    @everywhere α1 = $α1
    @everywhere S = HyperellipticSurface(gaps[1:g,:],some_zs[1:g,:],α1);
    @everywhere BA = BakerAkhiezerFunction(S,100.,1e-9,100,4,false);
    #  @everywhere BA = BakerAkhiezerFunction($S,200000.,1e-13,100,10,false);
    u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2)) - 0*h/2
    x = .1:.01:3 |> Array
    t = 0.0*pi
    U = pmap(x -> u(x,t) |> real, x);
    data[i] = maximum(abs.(U - map(x -> x > pi ? 1 : 0,x)))
    println((g,data[i]))
    i += 1
end

In [ ]:
data

In [ ]:
100:100:900 |> Array

In [ ]:
plot(100:100:900,data,yaxis = :log, xaxis = :log)

In [ ]:
@everywhere g = 300

In [ ]:
some_zs = hcat(zs[1:g] .- α1,sign.(map(tp,zs[1:g]) - map(tm,zs[1:g]) |> real));

In [ ]:
@everywhere some_zs = $some_zs
@everywhere gaps = $gaps
@everywhere α1 = $α1

In [ ]:
@everywhere S = HyperellipticSurface(gaps[1:g,:],some_zs[1:g,:],α1);

In [ ]:
@everywhere BA = BakerAkhiezerFunction(S,10.,1e-6);

In [ ]:
# shft = 0;
# u = (x,t) -> -KdV(BA,x/sqrt(6) - shft/(sqrt(6))*t,t*6^(-3/2)) - shft

In [ ]:
u = (x,t) -> -KdV(BA,x/sqrt(6) - 0*h/(2sqrt(6))*t,t*6^(-3/2)) - 0*h/2

In [ ]:
gaps[end-1,:][2] - gaps[end-1,:][1]

### g = 500

In [ ]:
x = 0:.01:2*pi |> Array
t = 0.1*pi
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p0 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-0.4,1.2],lw=3, label = @sprintf("q(x,t), t = %1.2f",t), framestyle = :box, fill = (-2,:lightblue))

In [ ]:
savefig(p0,"p0.pdf")

In [ ]:
x = 1:.001:2 |> Array
t = 0.1*pi
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p1 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.3,-.1],lw=3,label = @sprintf("q(x,t), t = %1.2f",t), framestyle = :box, fill = (-2,:lightblue))


In [ ]:
savefig(p1,"p1.pdf")

In [ ]:
x = 0:.01:2*pi |> Array
t = 0.0
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p2 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-0.4,1.2],lw=3, label = @sprintf("q(x,t), t = %1.2f",t), framestyle = :box, fill = (-2,:lightblue))

In [ ]:
savefig(p2,"p2.pdf")

In [ ]:
x = 0:.01:2*pi |> Array
t = 50.1*pi
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p3 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-1.4,2.2],lw=3,label = @sprintf("q(x,t), t = %1.2f",t), framestyle = :box, fill = (-2,:lightblue))

In [ ]:
savefig(p3,"p3.pdf")

In [ ]:
x = 0:.01:2*pi |> Array
t = 0.1
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
p4 = plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-1.4,2.2],lw=3,label = @sprintf("q(x,t), t = %1.2f",t), framestyle = :box, fill = (-2,:lightblue))

In [ ]:
savefig(p4,"p4.pdf")

# g = 300

In [ ]:
x = 0:.01:2*pi |> Array
t = 0.1*pi
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-0.4,1.2],lw=3,label = @sprintf("q(x,t), t = %1.2f",t), framestyle = :box, fill = (-2,:lightblue), legend = false)

In [ ]:
x = 1:.001:2 |> Array
t = 0.1*pi
U = pmap(x -> u(x,t) |> real, x);

In [ ]:
plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.3,-.1],lw=3,label = @sprintf("q(x,t), t = %1.2f",t), framestyle = :box, fill = (-2,:lightblue))


In [ ]:
x = 0:0.01:.5 |> Array
t = 0.25*pi
U = map(x -> u(x,t) |> real, x);

In [ ]:
plot(x, U, xaxis = [minimum(x),maximum(x)], yaxis = [-.2,.2],lw=3,label = @sprintf("q(x,t), t = %1.2f",t), framestyle = :box, fill = (-2,:lightblue))